# MS141 Lecture 12 

# Higher Order ODEs

## Read: Chapter 8 of Newman's book.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

# Set common figure parameters
newparams = {'figure.figsize': (10, 6), 'axes.grid': True,
             'lines.linewidth': 1.5, 'lines.markersize': 10,
             'font.size': 16}
plt.rcParams.update(newparams)

## 1. Second (or higher) order ODEs
The standard approach to solve ODEs with order 2 or higher is to *reduce them to a system of first order ODEs*:

$$ \frac{d\mathbf{y}}{dt} = f(t,\mathbf{y}),~~~~~\,~~~~\mathbf{y}(0) = \mathbf{y}_0.$$

In this form, we have an initial value system of first-order ODEs, and all the methods (Euler, Runge-Kutta, etc) we have studied so far can be applied by time-stepping all the equations simultaneously. 
The additional equations due to the higher order do not complicate the problem. 

Let's see how this works in practice. Suppose we want to solve the second order ODE (with two initial conditions):

$$ y''(t) = -\, \omega^2 y(t) ~~~~,~~~~ y(0) = 0~~~~,~~~ y'(0) = 1 .$$

with solution is $y(t) = \omega^{-1}\,\sin (\omega t)$. 

Let us denote by $y^{(i)}$ the $i$-th derivative. 
We set $ y^{(0)} = y$ and $ y^{(1)} = dy/dt $, and rewrite our second order ODE as a system of first order ODEs:

$$
\begin{align}
     &\frac{d}{dt} \begin{pmatrix}
           y^{(0)} \\
           y^{(1)} \\
%           \vdots \\
%           x_{m}
         \end{pmatrix}
         =          \begin{pmatrix}
           y^{(1)} \\
           -\omega^2 y^{(0)}
%           f(\,y^{(0)},y^{(1)},t\,) \\
         \end{pmatrix}
\end{align}
$$

This equation can be solved with the methods and codes from our last lecture on first order ODEs, provided we let $y$ be a vector with two components.<br>
Let's write the code to solve it using the Euler method, remembering to use a small enough time step.

In [ ]:
# function f(t,y) (f and y are vectors)
omega = 10.0

# RHS derivative function at current step (Euler)
def f(y0,y1):
    f0 = y1
    f1 = -1.0*omega**2 * y0
    # f0, f1
    return (f0,f1)

# Euler method 
N = 630 # number of steps
h = 0.001  # time step

t = np.zeros(N+1)
for i in range(N+1):
    t[i] = i*h
    
y0 = np.zeros(N+1)
y1 = np.zeros(N+1)


# initial values
t[0] = 0.0
y0[0] = 0.0
y1[0] = 1.0

for n in range(N):
    #time-step each component
    f0, f1 = f(y0[n],y1[n])

    y0[n+1] = y0[n] + h*f0 # this is our solution y(t)
    y1[n+1] = y1[n] + h*f1
    
    # debug
    #print(y0[n+1])

#print(r'y_N = %f' % y_old)

#Plot y0(t)
plt.plot(t, y0, 'b-', label='Euler')

y_exact = (1/omega)*np.sin(omega*t)
plt.plot (t, y_exact, '-r',label='Exact')

plt.ylabel(r'$y(t)$')
plt.xlabel(r'$t$')

plt.legend()
plt.grid()
plt.show()

The approach taken above to turn an $n$th order ODE into a system of first order ODEs can be generalized to higher orders.

### 1.1 Newton's equations

Classical mechanics makes extensive use of computational methods to solve the Newton equation, $\mathbf{F} = m\, \mathbf{a}$. Since the acceleration is the second derivative of the position, we can work with the position $\mathbf{x}(t)$ and the velocity $\mathbf{v}(t) = d \mathbf{x} / dt$ (either in one or multiple dimensions) using the formalism discussed above.

Restricting our analysis to one dimension, we write Newton's equation for a particle of mass $m$ as:

$$ m\frac{d^2}{dt^2}x(t)=F(v(t),x(t),t) $$

where the righthand side is the force. We can rewrite this second order ODE as two coupled first-order ODEs:

\begin{eqnarray*}
\frac{dx}{dt} &=& v, \\
\frac{dv}{dt} &=& \frac{F(v,x,t)}{m}.
\end{eqnarray*}

Similar to the example above, the time advancing Euler algorithm reads:

\begin{eqnarray*}
x_{n+1} &=& x_n+h\cdot v_n, \\
v_{n+1} &=& v_n+h\cdot \frac{F(v,x,t)}{m}.
\end{eqnarray*}  

We specify the initial conditions $x_0=x(t_0)$ and $v_0=v(t_0)$, and obtain a well-defined initial value problem.

#### Example.
Let's work out an example adapted from [NumFys](https://nbviewer.jupyter.org/urls/www.numfys.net/media/notebooks/eulers_method.ipynb). 
We consider a particle of mass $m$ in free fall toward the center of a planet of mass $M$.<br>
We assume that the atmosphere exerts a drag force on the particle proportional to the square of the velocity, 

$$ F_{\mathrm{drag}}=D\,v^2 $$

where $D$ is the drag coefficient. Note that the $x$-axis points away from the planet, so that $v\leq 0$ throughout the motion.<br> Using $G$ for the gravitational constant, Newton's equation of motion reads:

$$\frac{d^2 x}{dt^2}=\frac{D}{m}v^2-\frac{GM}{x^2}.$$

Using the method discussed above, we rewrite it as two coupled first order ODEs:

\begin{eqnarray*}
\frac{dx}{dt} &=& v, \\
\frac{dv}{dt} &=& \frac{D}{m}v^2-\frac{GM}{x^2}.
\end{eqnarray*} 

Accordingly, our algorithm becomes
\begin{eqnarray*}
x_{n+1} &=& x_n+h \cdot v_n, \\
v_{n+1} &=& v_n+h \cdot \left[  \frac{D}{m}v_n^2-\frac{GM}{x_n^2} \right] .
\end{eqnarray*}  

We simulate a body with mass $m=1\,\mathrm{kg}$ falling toward the center of the Earth 
($M=M_\mathrm{Earth}$ is the mass of the Earth), and use $D=0.0025\,\mathrm{kg}\,\mathrm{m}^{-1}$.<br>
The initial conditions and step size are 

$$ ~~~~x(t_0) = x_0 = 7000.0\,\mathrm{km},$$

$$\!\!\!\!\!\! v(t_0)=v_0 =0\,\mathrm{m/s}, $$

$$\!\!\!\!\!\!\!\!\!\!\!\!\!\!\!\!\!\!\!\! h = 0.001\,\mathrm{s}. $$

We can now simulate the dynamics until the particle hits the ground (inside the Earth, the equation of motion would be different!),<br> 
i.e. until $x=R_\mathrm{Earth}$, where $R_\mathrm{Earth}$ is the radius of Earth (roughly 6,350 km). 

This occurs in finite time both in reality and in the code. Let us integrate until $t_N=100\,\mathrm{s}$, and thus for $N=10^5$ time steps.<br> We will find that the particle has not yet reached the ground by $t=t_N$, and that the velocity stops increasing due to the drag term.

In [ ]:
# SI units
G = 6.67300e-11 # gravitational constant
M = 5.97219e24  # mass of Earth
D = 0.0025      # drag coefficient   
m = 1.0         # mass of particle

N = 100000      # number of steps
h = 0.001       # step size (0.001 s)

# initial values
t_0 = 0.0
x_0 = 7.0e6
v_0 = 0.0

t = np.zeros(N+1)
x = np.zeros(N+1)
v = np.zeros(N+1)
t[0] = t_0
x[0] = x_0
v[0] = v_0

for n in range(N):
    # Euler's method
    x_new = x[n] + h*v[n]
    v_new = v[n] + h*((D/m)*v[n]**2-G*m*M/x[n]**2)
    
    t[n+1] = t[n] + h
    x[n+1] = x_new
    v[n+1] = v_new

plt.figure()
plt.plot(t,x)  # plotting the position vs. time: x(t)
plt.xlabel(r'$t$')
plt.ylabel(r'$x(t)$')
plt.grid()

plt.figure()
plt.plot(t,v)  # plotting the velocity vs. time: v(t)
plt.xlabel(r'$t$')
plt.ylabel(r'$v(t)$')
plt.grid();

### 1.2 The Verlet method for Newton's equations

As classical mechanics is a very important application of scientific computing, several methods have been developed to time step Newton's equation. Splitting Newton's equation into two ODEs is not the only approach, and in fact perhaps not the most common.

For molecular dynamics or related particle-based simulations, in which one solves Newton's equations for a large number of particles, it's more convenient to find a direct method to time step the particle position that is a trade-off between accuracy and efficiency. One such algorithm is the [Verlet method](https://en.wikipedia.org/wiki/Verlet_integration).

In the Verlet method, one computes directly the second derivative of the position, using the central difference formula:

$$ \frac{x_{n+1} - 2 x_n + x_{n-1}}{h^2} = a(x_n)$$

where $a(x_n)$ is the acceleration. This approach defines the Verlet advancing scheme:

$$ \boxed{x_{n+1} = 2 x_n - x_{n-1}  + a_nh^2} + \mathcal{O}(h^4)$$

which has a local error of order $h^4$ in a single time step, and thus of order $\sim h^3$ after $N$ time steps. Verlet is therefore significantly more accurate than Euler in spite of a similar computational cost. Note that Verlet is a multi-step algorithm that requires data from two previous steps. The challenge is that the data doesn't exist on the first step $-$ we have $x_0$ and $v_0$ from the initial conditions, but clearly miss $x_{-1}$ at $t<0$, which is needed to compute $x_1$. 

To make this first step, we need to *bootstrap* the Verlet algorithm, and compute $x_1$ with a different method. Since we have the position and velocity at time zero, we can compute $x_1$ using:

$$ x_1 = x_0 + v_0\,h + \frac{1}{2} a \, h^2$$

which has an error of order $h^3$, and thus the same as the global error in the Verlet method. After obtaining $x_1$ this way, we can obtain $x_2$ and all the following steps using the Verlet formula. While the velocity is not needed in the Verlet method, it can also be computed using the central difference formula:

$$ v_n = \frac{x_{n+1} - x_{n-1}}{2h} + \mathcal{O}(h^2)$$

A variant of the Verlet method with increased stability is the widely employed *velocity Verlet* method, in which both the position and the velocity are updated simultaneously, with the velocity updated using the average force during the time step. Note that what Newman's book calls Verlet is in fact the velocity Verlet method.

Verlet methods are typically used only when velocity-dependent (e.g., friction) forces are not present, and thus energy is conserved. The leapfrog method, which is similar to velocity Verlet although it computes the velocity at the half-step, can be extended to include velocity-dependent forces with error $\mathcal{O}(h^4)$. 

The implementation of the Verlet method is simple and can be readily generalized to more than one dimension. Let's apply it to the problem we examined above, but leaving out the friction term.

In [ ]:
G = 6.67300e-11 # gravitational constant
M = 5.97219e24  # mass of Earth
D = 0.0025      # drag coefficient   
m = 1.0         # mass of particle

N = 12000      # number of steps
h = 0.001       # step size

# initial values
t_0 = 0.0
x_0 = 7.0e6
v_0 = 0.0

t = np.zeros(N+1)
x = np.zeros(N+1)

t[0] = t_0
x[0] = x_0

# Bootstrap the first step
x[1] = x_0 + v_0*h + -G*m*M/x_0**2

for n in range(1,N):
    # Verlet
    x[n+1] = 2*x[n] - x[n-1] + h*(-G*m*M/x_0**2)
    t[n+1] = t[n] + h

plt.figure()
plt.plot(t,x/1.e3,'r-')  # plotting the position vs. time: x(t)
plt.xlim(0.0,12.0)
plt.xlabel('time (s)')
plt.ylabel('Distance (km)')
plt.hlines(6.35e3, -0.05, 12.0, linestyles='--')
plt.grid()

Note how without friction the body reaches Earth's surface in roughly $t_N = 11.5$ s.  

## 2. Boundary value ODEs

Higher order ODEs in which the independent variable $x$ represents space rather than time are typically solved using methods different from those used for initial value problems we discussed for first and higher order ODEs in time.

In a typical scenario, one deals with a second order ODE, in which the solution $y(x)$ (or its derivatives) are fixed at the boundaries $x_1$ and $x_2$ of the solution domain. This type of physical problem, which typically leads to an eigenvalue equation, is known as a *boundary value problem (BVP)*. A simple example is the equation:

$$ \frac{d^2 y}{dx^2} + k^2 y = 0 ~~~~,~~~~ y(0) = y(a) = 0,~~~~~~~ k>0 . $$

solved in the $0 \leq x \leq a$ domain, with solution $y(x) = \sin(\frac{n\pi}{a}x)$, where $n = 1,2,\ldots$ is an integer, and $k_n = \left( \frac{n\pi}{a} \right)^2$.

Two common methods for solving BVP are the [shooting method](https://en.wikipedia.org/wiki/Shooting_method) and the [finite difference method for ODEs](https://en.wikipedia.org/wiki/Finite_difference_method#Example:_ordinary_differential_equation). The shooting method (see Newman's book) recasts the BVP into a (fictitious) initial value problem, performing a search with a root-finding method of the initial value conditions that allow one to satisfy the boundary conditions. The shooting method is simple but is a rather back-handed way of solving boundary value ODEs, and thus we will not examine it further. It also cannot be generalized to boundary value problems in multiple dimensions, so it cannot be applied to PDEs. 

### 2.1 Finite difference method for boundary value ODEs

The *finite difference method* is a general framework for solving boundary value ODEs, and it can be generalized to multiple dimensions, and thus it is also widely used also for PDEs. It is perhaps the most useful approach for numerical differential equations. For this reason, we'll focus here on the finite difference method. 

Let us assume we have a boundary value problem of the form 

$$ \frac{d^2y}{dx^2} = g(x)~~~~~~,~~~~~~ y(x_L) = y_L,~~~ y(x_R) = y_R  $$

with boundary condition $y_L$ at the left end $x_L$ and $y_R$ at the right end $x_R$ of the solution domain.<br>
We first set up a uniform grid for the spatial variable, with $N$ points $x_n = n\,h$, with $n=1,\ldots N$, and $x_1 = x_L$ and $x_N = x_R$. 
We discretize the second order derivative at each point $n$, writing it with the central difference formula:

$$ \left(\frac{d^2y}{dx^2}\right)_n \approx \frac{y_{n+1} - 2 y_n + y_{n-1}}{h^2} $$

When applied to the entire grid, the second derivative operator can be written as an $N \times N$ *tridiagonal* matrix:

\begin{equation}  
\begin{pmatrix}
           (d^2y/dx^2)_1 \\
          (d^2y/dx^2)_2 \\
          \vdots \\
          (d^2y/dx^2)_n \\
          \vdots \\
          (d^2y/dx^2)_N
         \end{pmatrix}
= \frac{1}{h^2} \left[
\begin{matrix}
\ddots & \ddots & 0 & \ldots & 0 & 0\\
1 & -2 & 1 & 0 & \ldots & 0 \\
0 & \ddots & \ddots & \ddots & 0 & 0\\
      0  & 0 & \ddots & \ddots & \ddots & 0 \\
0  & \ldots & 0 &  1 & -2 & 1\\
  0 & 0 & \ldots & 0 & \!\!\!\ddots & \!\!\!\ddots  \end{matrix}  \right]
\,\,\begin{pmatrix}
           y_1 \\
          y_2 \\
          \vdots \\
          y_n \\
          \vdots \\
          y_N
         \end{pmatrix}
\end{equation}

This matrix is a basic example of a *stencil* used to represent differential operators on a grid. We have left out the matrix entries in the first and last row because we can use them to **apply the boundary conditions**.
To this end, we write the equation above, $y''(x) = g(x)$, as:

\begin{equation}  
\left[
\begin{matrix}
-2 & 0 & 0 & \ldots & 0 & 0\\
1 & -2 & 1 & 0 & \ldots & 0 \\
0 & \ddots & \ddots & \ddots & 0 & 0\\
      0  & 0 & \ddots & \ddots & \ddots & 0 \\
0  & \ldots & 0 &  1 & -2 & 1\\
  0 & 0 & \ldots & 0 & \!\!\!0 & \!\!\!-2  \end{matrix}  \right]
\,\,\begin{pmatrix}
           y_1 \\
          y_2 \\
          \vdots \\
          \vdots \\
          y_{N-1} \\
          y_N
         \end{pmatrix} 
   = \,\,\begin{pmatrix}
          -2\,y_L \\
          g_2h^2 \\
          \vdots \\
          \vdots \\
          g_{N-1} h^2 \\
          -2 y_R
         \end{pmatrix}
\end{equation}

We can see from the first and last row that the boundary conditions are automatically satisfied when the problem is written in this form. An alternative approach, which we'll follow below, is to use grid point indices running only from 2 to $N-1$, writing the equation as a $(N-2) \times (N-2)$ matrix in which the boundary value has been moved to the vector in the righthand side:

\begin{equation}  
\left[
\begin{matrix}
 -2 & 1 & 0 & \ldots & 0 \\
\ddots & \ddots & \ddots & 0 & 0\\
      0  & 1 & -2 & 1 & 0 \\
0  & 0 &  \ddots & \ddots & \ddots \\
  0 & 0 & 0 & \!\!\!1 & \!\!\!-2  \end{matrix}  \right]
\,\,\begin{pmatrix}
          y_2 \\
          \vdots \\
          y_n \\
          \vdots \\
          y_{N-1}
         \end{pmatrix} 
   = \,\,\begin{pmatrix}
          g_2h^2 - y_L\\
          \vdots \\
          g_n h^2 \\
          \vdots \\
          g_{N-1} h^2 - y_R\\
         \end{pmatrix}
\end{equation}

This tridiagonal sparse matrix is symmetric, which is a great advantage for solving linear systems as one can perform Cholesky decomposition. We have successfully discretized our boundary value problem, casting it in the form of a linear system:

$$A\mathbf{y} = \mathbf{g}.$$ 

**Example.** 
We will now apply this approach to solving the time-independent Schroedinger equation for a particle in an infinite potential well of size $a$ (the so-called "particle in a box"). 
We write the Schrodinger equation for this problem as:

$$ -\frac{\hbar^2}{2m} \frac{d^2\Psi(x)}{dx^2} + V(x)\, \Psi(x) \,=\, E\, \Psi(x)$$

where $\Psi$ is the particle wavefunction, $m$ its mass and $E$ its energy ($\hbar$ is Planck's reduced constant). 
In the equation, we take the potential $ V(x) = 0$ inside the box, and thus for $0 \leq x \leq a$, and $V(x) = \infty$ outside the box ( $x < 0$ and $x>a$ ). The boundary conditions to impose for this problem are that the wavefunction $\Psi$ vanishes at the two ends of the box, while the Schrodinger equation is solved inside the box with a zero potential. We thus arrive at the final form of our equation:

$$ \frac{d^2\Psi(x)}{dx^2} + k^2 \Psi(x) = 0,~~~~~~~~~\Psi(0) = \Psi(a) = 0 $$

where $k^2 = \frac{2mE}{\hbar^2}$. As we mentioned above, we know that the exact solution to this problem is 

$$\Psi(x) = \left(\frac{2}{a}\right)^{1/2} \sin\left(\frac{n\pi}{a}x \right) $$

where $n = 1,2,\ldots$ is an integer, and the constant multiplying the sine function is employed to normalized the wavefunction. Starting from the corresponding eigenvalues $k_n^2 = \left( \frac{n\pi}{a} \right)^2 = \frac{2 m E_n}{\hbar^2}$, we find the energies of the particle in a box:

$$ E_n = \frac{\hbar^2\pi^2}{2m a^2} n^2$$

We will find these results by solving the equation numerically in the spatial region $0 \leq x \leq a$, in which we set up a spatial grid with $N$ points. Once the Hamiltonian matrix has been discretized, we diagonalize it using the `scipy.linalg` wrapper for the LAPACK [`ssbevd`](http://www.netlib.org/lapack/explore-html/d7/d93/group__real_o_t_h_e_reigen_ga40e8f9c2d9853f400715b94a9c5225ab.html) algorithm, which computes the eigenvalues and eigenvectors of real symmetric band matrices (another name for tridiagonal matrices).

In [ ]:
# Solving (-hbar^2 / 2m) d^2 Psi / dx^2 = E \Psi in [0,a]
# by finding the eigenvalues / eigenvectors of the lefthand side
# Impose Psi(0) = Psi(a) = 0 using an N-1 by N-1 matrix

# routine to diagonalize the tridiagonal matrix
from scipy.linalg.lapack import ssbevd 

# work in SI units for clarity (though atomic units would be easier)
hbar = 1.05e-34  # J⋅s. Reduced Plank's constant 
m = 9.11e-31     # kg.  Electron mass 
ev = 1.6e-19 # eV

h = 1e-10       # m units; step size h = 1 Angstrom
N = 1000         # Number of grid points in the well region
a = h*N         # well length
x = np.linspace(0, a, N)
psi = np.zeros((N,N))

# Potential
V = np.zeros(N-2)

# diagonal and superdiagonal (adjacent to diagonal) entries
# solve in the interior and store in psi_int
diag = np.ones(N-2) * 2.0 * hbar**2/(2*m*h**2) + V #Diagonal
sup_diag = np.ones(N-2)*(-hbar**2/(2*m*h**2)) # Superdiagonal
E, psi_int, _ = ssbevd([sup_diag, diag]) # Call solver

# impose the BCs at 1 and N
# psi stores the wfn:  psi[:,i] is the i-th eigenvector
for i in range(N-2):
    psi[1:N-1,i] = psi_int[:,i]

# debug
# print (psi[0,0])
# print (psi.shape)
# print (E[0]/ev, E[1]/ev)

Now we plot the wavefunctions and the eigenvalues

In [ ]:
# Plot the eigenstates
newparams = {'axes.labelsize': 25, 'axes.linewidth': 1, 'savefig.dpi': 200,
             'lines.linewidth': 3, 'figure.figsize': (20, 10),
             'ytick.labelsize': 25, 'xtick.labelsize': 25,
             'ytick.major.pad': 5, 'xtick.major.pad': 5,
             'figure.titlesize': 25, 
             'legend.fontsize': 25, 'legend.frameon': True, 
             'legend.handlelength': 1.5, 'axes.titlesize': 25,
             'mathtext.fontset': 'stix',
             'font.family': 'STIXGeneral'}
plt.rcParams.update(newparams)

my_label=['n = 1','n = 2','n = 3', 'n = 4','n = 5']
for i in [0, 1, 3]:
    plt.plot(x,psi[:,i], label=my_label[i])
    
plt.xlabel("$x$ (m)")
plt.ylabel("$\psi_n(x)$")
plt.ylim(-0.1,0.1)
plt.xlim(0.,h*N)
plt.legend()
plt.show();

In [ ]:
# Plot the energies for a few states

newparams = {'figure.figsize': (10, 6), 'axes.grid': True,
             'lines.linewidth': 1.5, 'lines.markersize': 10,
             'font.size': 16}
plt.rcParams.update(newparams)

x_fict = np.linspace(0,1.0,1000)
#my_label=['n = 1','n = 2','n = 3', 'n = 4','n = 5']
for i in range(5):
    plt.plot(x_fict,E[i]*np.ones(1000)/E[0], label=(i+1)) 
plt.ylabel("Energy (units of " r'$E_1$' ")")
plt.xticks([])
plt.xlim(0.,h*N)
plt.legend()
plt.show();